# Assignment 6: Boston House Prediction using DNN


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn

In [ ]:
ds = pd.read_csv( "HousingData.csv" )
ds

In [ ]:
ds.dropna(inplace=True)

In [ ]:
def min_max_normalize(name):
  max_ = ds[name].max()
  min_ = ds[name].min()
  ds[name] = (ds[name] - min_) / ( max_ - min_ )

for col in ds.drop(["MEDV"], axis=1).columns:
    min_max_normalize(col)

In [ ]:
from sklearn.model_selection import train_test_split

x = np.array(ds.drop(["MEDV"], axis=1))
y = np.array(ds["MEDV"])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=22)
y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

x_train = torch.Tensor(x_train)
y_train = torch.Tensor(y_train)
x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test)

loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=16, shuffle=True)

In [ ]:
model = nn.Sequential(
    nn.Linear(13, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

loss = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_step(batch_x, batch_y):
  model.train()
  preds = model.forward(batch_x)
  batch_loss = loss(preds, batch_y)
  batch_loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  return batch_loss.item()

def train_epoch():
  epoch_loss = 0.0
  for batch_x, batch_y in loader:
    epoch_loss += train_step(batch_x, batch_y)
  return epoch_loss / len(loader)

def val_epoch():
  model.eval()
  preds = model.forward(x_test)
  val_loss = loss(preds, y_test)
  return val_loss.item()

for e in range(200):
  train_loss = train_epoch()
  val_loss = val_epoch()
  print(train_loss, val_loss)